We’ll build a Multi-Skill Intelligent Research Agent that can:

 ✅ read and analyze text<br>
 ✅ perform web search (real-time using DuckDuckGo 🦆)<br>
 ✅ do math reasoning (like currency conversions or stats)<br>
 ✅ summarize and cross-combine results<br>

🚀 Goal

You’ll be able to ask something like:<br>

“Find the latest news about AI regulation in Europe, summarize it, and calculate how many years since the first major EU AI proposal in 2021.”

In [2]:
from langchain_community.llms import Ollama
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
import math, re, datetime

1. Connect to local model

In [3]:
llm = Ollama(model="mistral")  # you can replace with llama3, phi3, etc.

/tmp/ipykernel_50649/1810637363.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")  # you can replace with llama3, phi3, etc.


2. Define custom tools

In [21]:
@tool
def summarize_text(text: str) -> str:
    """Summarizes a long text into key points."""
    sentences = re.split(r'(?<=[.!?]) +', text)
    summary = " ".join(sentences[:5])
    return f"Summary: {summary}"

@tool
def simple_calculator(expression: str) -> str:
    """Safely evaluates a basic math expression."""
    expr = expression.lower()
    expr = expr.replace("sqrt", "math.sqrt").replace("^", "**")
    try:
        result = eval(expr, {"__builtins__": None, "math": math})
        return str(result)
    except Exception as e:
        return f"Error evaluating: {e}"

@tool
def years_since_contextual(input_text: str) -> str:
    """
    Calculates years since a given year, considering contextual text like 'news from 2025'.
    Input should look like: 'year=2021; context=news from 2025'
    """
    try:
        # Extract numbers (years)
        years = [int(y) for y in re.findall(r"20\d{2}", input_text)]
        if not years:
            return "No valid year found."

        target_year = years[0]
        ref_year = years[-1] if len(years) > 1 else datetime.datetime.now().year

        diff = ref_year - target_year

        if diff < 0:
            return f"The year {target_year} is after {ref_year}."
        elif diff == 0:
            return f"No years have passed between {target_year} and {ref_year}."
        else:
            return f"{diff} years have passed between {target_year} and {ref_year}."
    except Exception as e:
        return f"Error calculating years: {e}"



3. Use community search tool

In [22]:
web_search = DuckDuckGoSearchRun()

Add all tools

In [23]:
tools = [summarize_text, simple_calculator, years_since_contextual, web_search]

4. Initialize the Agent

In [24]:
agent = initialize_agent(
    tools,
    llm,
    agent_type=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

5. Ask a complex question

In [25]:
query = """
Search for the latest 2025 news about AI regulation in Europe. 
Summarize the findings in a few sentences.
Then calculate how many years have passed since 2021.
"""

result = agent.run(query)

print("\n🧠 Final Combined Result:\n", result)



> Entering new AgentExecutor chain...
 I need to search for the latest news about AI regulation in Europe from 2025, summarize the findings, and then calculate the number of years that have passed since 2021.

Action: duckduckgo_search
Action Input: "latest news AI regulation Europe 2025"
Observation: The Artificial Intelligence Act is a European Union regulation concerning artificial intelligence. It establishes a common regulatory and legal framework for AI within the European Union. It came into force on 1 August 2024... July 10, 2025 - The so-called code of practice ... year. Rules for general-purpose A.I. systems take effect on Aug. 2, though E.U. regulators will not be able to impose penalties for noncompliance until August 2026 , according to the European Commission, the executive branch of the 27-nation ... May 2, 2025 - Anna-Lena Kempf, also of Pinsent Masons, said, though, that “ Article 56 of the EU AI Act obliges the AI Office to develop and publish so-called Codes of Pra